# Connecting to the Database

In [1]:
import sqlalchemy
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'officems_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'officems_test'
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# Create the connection
cnx = create_engine(postgres_str)

# with cnx.connect() as conn:
conn = cnx.raw_connection()
try:
    cursor = conn.cursor()
    cursor.execute("select version()")

    # Fetch a single row using fetchone() method.
    data = cursor.fetchone()
    print("Connection established to: ",data)
    #Retrieving data
    cursor.execute('''SELECT id, parent_id, name_dr from DEPARTMENT''')

    #Fetching 1st row from the table
    result = cursor.fetchall();
    #closing the connection
    conn.close()
finally:
    conn.close()


# db_name = 'officems_test'
# user_name = 'officems_user'
# password = 'secret'

# #establishing the connection
# conn = psycopg2.connect(
#    database=db_name, user=user_name, password=password, host='127.0.0.1', port= '5432'
# )
#Creating a cursor object using the cursor() method


#Executing an MYSQL function using the execute() method
# cursor.execute("select version()")

# # Fetch a single row using fetchone() method.
# data = cursor.fetchone()
# print("Connection established to: ",data)

# #Retrieving data
# cursor.execute('''SELECT id, parent_id, name_en from DEPARTMENT''')

# #Fetching 1st row from the table
# result = cursor.fetchall();
# #closing the connection
# conn.close()

Connection established to:  ('PostgreSQL 12.6, compiled by Visual C++ build 1914, 64-bit',)


# Converting the Results to Pandas DataFrame

In [2]:
import pandas as pd
import numpy as np

df = pd.DataFrame(result, columns = ["id", "parent_id", "name"])
df = df.fillna(0)
df['parent_id'] = df['parent_id'].astype('int')
# df

# Generating the Closure Table

In [3]:
nodes = []
j = 0
for i, record in df.iterrows():
    
    descending = int(record.id)
    ascending = int(record.parent_id)
    depth = 0
    j = j+1
    nodes.append((j, descending, descending, depth))
    
    while(ascending != 0):
        j = j+1
        depth = depth+1
        nodes.append((j, descending, ascending, depth))
        ascending = int(df[df['id'] == ascending]['parent_id'])
        

In [4]:
# print(nodes)

In [5]:
table = pd.DataFrame(nodes, columns = ['id', 'descendant_id', 'ancestor_id', 'depth'])
# table

In [6]:
table.to_sql('department_closure', con=cnx, if_exists='append',index=False)
print("successfully saved in db")

successfully saved in db
